# Bab 16: Pemrosesan Bahasa Alami dengan RNN dan Mekanisme Perhatian

**Tujuan Pembelajaran:**

Notebook ini bertujuan untuk memperdalam pemahaman dan keterampilan praktis dalam mengimplementasikan konsep inti Pemrosesan Bahasa Alami (NLP) menggunakan Recurrent Neural Networks (RNN) dan Mekanisme Perhatian, merujuk pada Bab 16 dari buku "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow."

---

### **Pendahuluan**

Bab ini akan membawa kita menyelami dunia Pemrosesan Bahasa Alami (NLP) yang menarik, khususnya dengan fokus pada Recurrent Neural Networks (RNN) dan mekanisme perhatian. Kita akan melihat bagaimana model-model ini dapat memahami, menghasilkan, dan menerjemahkan bahasa manusia.

---

### **1. Membangun dan Menghasilkan Teks ala Shakespeare dengan Character RNN**

#### **1.1. Membuat Dataset Pelatihan**

**Teori:**
Untuk melatih RNN agar dapat menghasilkan teks, kita memerlukan korpus teks yang besar. Dalam kasus ini, kita akan menggunakan karya-karya Shakespeare. Teks ini perlu diubah menjadi representasi numerik yang dapat dipahami oleh model. Pendekatan "Character RNN" (Char-RNN) berarti kita akan memprediksi karakter berikutnya dalam sebuah urutan. Setiap karakter akan diberi ID numerik unik. Keras's `Tokenizer` adalah alat yang sangat berguna untuk tugas ini.

In [1]:
# Import library yang diperlukan
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import time
from collections import Counter

# Pastikan TensorFlow dan Keras sudah terinstal dan berfungsi
print(tf.__version__)
print(keras.version)

# Download karya-karya Shakespeare
shakespeare_url = "https://homl.info/shakespeare" # URL shortcut dari buku
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)

# Baca teks
with open(filepath, 'r') as f:
    shakespeare_text = f.read()

print(f"Total karakter dalam teks Shakespeare: {len(shakespeare_text)}")

# Buat tokenizer tingkat karakter
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts([shakespeare_text])

# Informasi tokenizer
max_id = len(tokenizer.word_index) # Jumlah karakter unik
dataset_size = tokenizer.document_count # Total karakter
print(f"Jumlah karakter unik: {max_id}")
print(f"Total karakter (setelah tokenisasi): {dataset_size}")

# Contoh encoding dan decoding
print(f"Encoding 'First': {tokenizer.texts_to_sequences(['First'])}")
print(f"Decoding [[20, 6, 9, 8, 3]]: {tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])}")

# Encode seluruh teks ke dalam ID karakter (dimulai dari 0)
encoded = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
encoded_train = encoded[0] # Ambil array 1D
print(f"Bentuk encoded teks: {encoded_train.shape}")

2.18.0
<function version at 0x7dbd2471ed40>
1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Total karakter dalam teks Shakespeare: 1115394
Jumlah karakter unik: 39
Total karakter (setelah tokenisasi): 1
Encoding 'First': [[20, 6, 9, 8, 3]]
Decoding [[20, 6, 9, 8, 3]]: ['f i r s t']
Bentuk encoded teks: (1115394,)


#### **1.2. Cara Membagi Dataset Sekuensial (dan Memotongnya menjadi Jendela)**

**Teori:**
Dalam NLP, kita tidak bisa hanya mengacak semua karakter atau kata dalam teks karena akan menghancurkan struktur sekuensialnya. Untuk melatih RNN, kita perlu mempertahankan urutan. Oleh karena itu, pembagian dataset dilakukan berdasarkan urutan waktu. Misalnya, kita dapat mengambil 90% pertama dari teks untuk pelatihan, 5% berikutnya untuk validasi, dan 5% terakhir untuk pengujian.

Char-RNN melatih model untuk memprediksi karakter berikutnya dalam sebuah urutan. Untuk ini, kita perlu membuat "jendela" dari teks, di mana setiap jendela adalah substring pendek dari teks lengkap. Ini dikenal sebagai *truncated backpropagation through time* (BPTT). Metode `window()` dari `tf.data.Dataset` sangat cocok untuk ini, memungkinkan kita untuk membuat jendela yang tumpang tindih (`shift=1`) untuk memaksimalkan penggunaan data.

In [2]:
# Bagi dataset menjadi training, validation, dan test set
train_size = encoded_train.shape[0] * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded_train[:train_size])

# Tentukan panjang jendela (n_steps) dan buat jendela
n_steps = 100 # Panjang sekuens input yang akan diproses RNN
window_length = n_steps + 1 # Target adalah input yang digeser 1 karakter ke depan

# Buat jendela yang tumpang tindih
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

# Ratakan dataset bersarang menjadi dataset datar
dataset = dataset.flat_map(lambda window: window.batch(window_length))

# Acak jendela dan pisahkan input (n_steps pertama) dari target (karakter terakhir)
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

# One-hot encode input karakter
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

# Tambahkan prefetching untuk efisiensi
dataset = dataset.prefetch(1)

# Verifikasi bentuk dataset
for X_batch, Y_batch in dataset.take(1):
    print(f"Bentuk X_batch: {X_batch.shape}")
    print(f"Bentuk Y_batch: {Y_batch.shape}")

Bentuk X_batch: (32, 100, 39)
Bentuk Y_batch: (32, 100)


#### **1.3. Membangun dan Melatih Model Char-RNN**

**Teori:**
Model Char-RNN akan terdiri dari beberapa lapisan GRU (Gated Recurrent Unit) yang ditumpuk, diikuti oleh lapisan `TimeDistributed(Dense)`. Lapisan GRU sangat baik dalam menangani dependensi jangka panjang dalam sekuens. `return_sequences=True` penting untuk lapisan GRU kecuali yang terakhir jika kita ingin setiap langkah waktu menghasilkan output. Lapisan `TimeDistributed(Dense)` memungkinkan lapisan `Dense` diterapkan secara independen pada setiap langkah waktu dari sekuens input. Karena kita memprediksi salah satu dari `max_id` karakter unik, lapisan output akan memiliki `max_id` unit dengan fungsi aktivasi `softmax`. Fungsi *loss* yang umum untuk tugas klasifikasi multi-kelas dengan label integer sparse adalah `sparse_categorical_crossentropy`.

In [3]:
# Bangun model Char-RNN
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2), # max_id adalah dimensi one-hot
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

# Kompilasi model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# Ringkasan model
model.summary()

# Latih model (Epochs mungkin perlu disesuaikan tergantung pada kinerja)
# history = model.fit(dataset, epochs=20) # Uncomment untuk melatih

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, None, 128)      │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, None, 39)       │         5,031 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,999 (660.15 KB)

 Trainable params: 168,999 (660.15 KB)

 Non-trainable params: 0 (0.00 B)

#### **1.4. Menggunakan Model Char-RNN**

**Teori:**
Setelah model dilatih, kita dapat menggunakannya untuk memprediksi karakter berikutnya dari teks input. Fungsi `predict()` akan menghasilkan probabilitas untuk setiap karakter yang mungkin. Untuk menghasilkan teks yang lebih beragam dan menarik, kita tidak selalu memilih karakter dengan probabilitas tertinggi, melainkan mengambil sampel karakter secara acak berdasarkan distribusi probabilitas yang diprediksi. Konsep "suhu" diperkenalkan untuk mengontrol keragaman ini: suhu yang lebih rendah akan membuat model lebih percaya diri (kurang beragam), sementara suhu yang lebih tinggi akan membuat model lebih "kreatif" (lebih beragam).

In [4]:
# Fungsi pra-pemrosesan untuk teks baru
def preprocess_text(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

# Fungsi untuk memprediksi karakter berikutnya
def next_char(text, temperature=1):
    X_new = preprocess_text([text])
    y_proba = model.predict(X_new)[0, -1:, :] # Ambil probabilitas output dari langkah waktu terakhir
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

# Fungsi untuk melengkapi teks
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

# Contoh penggunaan (setelah model dilatih)
# print(f"Teks dihasilkan (suhu 0.2): {complete_text('t', temperature=0.2)}")
# print(f"Teks dihasilkan (suhu 1): {complete_text('w', temperature=1)}")
# print(f"Teks dihasilkan (suhu 2): {complete_text('w', temperature=2)}")

#### **1.5. RNN Berstateful**

**Teori:**
RNN stateless, seperti yang kita gunakan sejauh ini, menginisialisasi *hidden state* dengan nol di setiap iterasi pelatihan dan membuangnya setelah setiap *batch*. Ini berarti mereka hanya dapat belajar pola yang lebih pendek dari panjang sekuens yang dilatih. RNN stateful, di sisi lain, mempertahankan *hidden state* dari satu *batch* ke *batch* berikutnya. Ini memungkinkan model untuk belajar pola jangka panjang yang melampaui batas satu sekuens dalam *batch*. Kunci untuk RNN stateful adalah memastikan bahwa sekuens input dalam satu *batch* melanjutkan tepat di mana sekuens yang sesuai di *batch* sebelumnya berhenti. Ini memerlukan pengaturan `stateful=True` pada lapisan RNN dan menentukan `batch_input_shape` pada lapisan pertama.

In [5]:
# Konfigurasi dataset untuk RNN stateful (membutuhkan sekuens berurutan dan non-tumpang tindih)
# Ini lebih kompleks, jadi kita akan membuat dataset yang disederhanakan untuk demonstrasi

# Untuk demonstrasi, kita akan membuat dataset batch_size=1
# Dalam kasus nyata, perlu strategi pembagian dan batching yang lebih canggih.

seq_length_stateful = n_steps # Panjang sekuens untuk RNN stateful

dataset_stateful = tf.data.Dataset.from_tensor_slices(encoded_train[:train_size])
dataset_stateful = dataset_stateful.window(seq_length_stateful, shift=seq_length_stateful, drop_remainder=True)
dataset_stateful = dataset_stateful.flat_map(lambda window: window.batch(seq_length_stateful))
dataset_stateful = dataset_stateful.batch(1) # Batch size 1 untuk kesederhanaan
dataset_stateful = dataset_stateful.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset_stateful = dataset_stateful.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset_stateful = dataset_stateful.prefetch(1)

# Definisikan batch_size untuk model stateful (harus sama dengan batch_size dataset)
stateful_batch_size = 1

# Asumsi: max_id sudah terdefinisi dari kode sebelumnya (misalnya, 39)
# Asumsi: stateful_batch_size sudah terdefinisi (misalnya, 1)

# Dummy values for demonstration if not already defined
if 'max_id' not in locals():
    max_id = 39
if 'stateful_batch_size' not in locals():
    stateful_batch_size = 1

# Bangun model RNN stateful
stateful_model = keras.models.Sequential([
    # Gunakan Input layer terpisah untuk mendefinisikan batch_input_shape
    keras.layers.Input(batch_shape=[stateful_batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

# Callback untuk mereset state di awal setiap epoch
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

# Kompilasi model stateful
stateful_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# Ringkasan model stateful
stateful_model.summary()

# Latih model stateful (Epochs mungkin perlu lebih banyak karena batch_size kecil)
# history_stateful = stateful_model.fit(dataset_stateful, epochs=50,
#                                       callbacks=[ResetStatesCallback()]) # Uncomment untuk melatih

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                     │ (1, None, 128)         │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (1, None, 128)         │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (1, None, 128)         │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (1, None, 39)          │         5,031 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 268,071 (1.02 MB)

 Trainable params: 268,071 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

---

### **2. Analisis Sentimen (Sentiment Analysis)**

#### **Teori:**
Analisis sentimen adalah tugas NLP yang bertujuan untuk menentukan "nada" atau "emosi" di balik sebuah teks, biasanya diklasifikasikan sebagai positif atau negatif. Dataset ulasan film IMDb adalah contoh klasik untuk ini. Daripada memproses karakter, kita sekarang akan memproses kata. Kata-kata akan diubah menjadi ID numerik (seperti pada Char-RNN), dan kemudian sering kali diwakili sebagai *word embeddings* (yang akan dibahas lebih lanjut).

Salah satu tantangan dalam memproses teks adalah panjang sekuens yang bervariasi. Ulasan film bisa sangat panjang atau sangat pendek. Untuk mengatasi ini, kita dapat melakukan *padding* pada sekuens yang lebih pendek sehingga semua sekuens dalam satu *batch* memiliki panjang yang sama. Keras menyediakan mekanisme *masking* (`mask_zero=True` pada lapisan `Embedding`) untuk mengabaikan token *padding* ini selama pelatihan, sehingga model fokus pada konten yang relevan.

In [6]:
# Muat dataset ulasan film IMDb
(X_train_imdb, y_train_imdb), (X_test_imdb, y_test_imdb) = keras.datasets.imdb.load_data()

# Dapatkan kamus kata-ID
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()} # +3 karena ID 0, 1, 2 spesial
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

# --- Perbaikan dimulai di sini ---
# Tentukan panjang maksimum untuk padding
# Anda bisa memilih panjang ini berdasarkan distribusi panjang ulasan Anda
# Atau gunakan nilai default yang masuk akal, misalnya 300 seperti di bagian preprocess
max_review_length = 300

# Lakukan padding pada X_test_imdb
X_test_imdb_padded = keras.preprocessing.sequence.pad_sequences(X_test_imdb,
                                                                 maxlen=max_review_length,
                                                                 padding='post', # Padding di akhir sekuens
                                                                 truncating='post', # Truncate di akhir sekuens jika terlalu panjang
                                                                 value=0) # Value untuk padding (0 adalah token <pad>)

# Konversi NumPy array yang sudah di-padding ke tf.constant
# Kemudian bagi data validasi dan data test final
X_test_val_imdb = tf.constant(X_test_imdb_padded[:12500])
X_test_final_imdb = tf.constant(X_test_imdb_padded[12500:])

y_test_val_imdb = tf.constant(y_test_imdb[:12500])
y_test_final_imdb = tf.constant(y_test_imdb[12500:])

print("X_test_val_imdb shape:", X_test_val_imdb.shape)
print("y_test_val_imdb shape:", y_test_val_imdb.shape)
print("X_test_final_imdb shape:", X_test_final_imdb.shape)
print("y_test_final_imdb shape:", y_test_final_imdb.shape)

# --- Perbaikan berakhir di sini ---

# Lanjutkan dengan kode yang lain...

# Model untuk analisis sentimen (menggunakan vocab_size_imdb dan embed_size_imdb)
# Pastikan variabel ini terdefinisi jika Anda menjalankan potongan kode ini secara terpisah
# Contoh dummy jika belum terdefinisi:
vocab_size_imdb = 10000
num_oov_buckets_imdb = 1000
embed_size_imdb = 128

sentiment_model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size_imdb + num_oov_buckets_imdb, embed_size_imdb,
                           input_shape=[None], mask_zero=True), # mask_zero=True untuk padding
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid") # Output biner (positif/negatif)
])

# Kompilasi model
sentiment_model.compile(loss="binary_crossentropy", optimizer="adam",
                        metrics=["accuracy"])

# Ringkasan model
sentiment_model.summary()

# Latih model
# history_sentiment = sentiment_model.fit(train_set_imdb, epochs=5) # Uncomment untuk melatih

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X_test_val_imdb shape: (12500, 300)
y_test_val_imdb shape: (12500,)
X_test_final_imdb shape: (12500, 300)
y_test_final_imdb shape: (12500,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 128)      │     1,408,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, None, 128)      │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,606,273 (6.13 MB)

 Trainable params: 1,606,273 (6.13 MB)

 Non-trainable params: 0 (0.00 B)

#### **2.1. Menggunakan Embedding yang Sudah Dilatih (Pretrained Embeddings)**

**Teori:**
Melatih *word embeddings* dari awal membutuhkan korpus teks yang sangat besar. Untungnya, kita dapat menggunakan *embeddings* yang sudah dilatih sebelumnya dari korpus umum yang lebih besar (misalnya, Wikipedia atau Google News). *Embeddings* ini sering kali sudah menangkap banyak informasi semantik tentang kata-kata dan dapat secara signifikan meningkatkan kinerja model, terutama ketika dataset pelatihan kita kecil. TensorFlow Hub menyediakan cara mudah untuk mengunduh dan menggunakan *modules* model yang sudah dilatih.

In [10]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import numpy as np
from collections import Counter
import tensorflow_datasets as tfds # Untuk memuat dataset imdb

# --- Muat dataset IMDb (jika belum dimuat di sesi ini) ---
# Ini penting agar datasets_imdb dan info_imdb tersedia
try:
    datasets_imdb, info_imdb = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
    train_size_imdb = info_imdb.splits["train"].num_examples
except tfds.core.ReadError:
    print("Dataset IMDb tidak dapat dimuat, pastikan sudah diunduh atau jalankan di Colab.")


# --- Solusi: Perbaiki WrappedHubLayer untuk menangani output_shape dengan benar ---
class WrappedHubLayer(keras.layers.Layer):
    def __init__(self, hub_url, **kwargs):
        # Ambil output_shape dari kwargs sebelum meneruskannya ke super()
        # dan simpan sebagai atribut jika Anda berencana menggunakannya.
        # Namun, lebih baik membiarkan hub.KerasLayer internal menentukannya.
        # Argumen 'output_shape' sebenarnya adalah untuk hub.KerasLayer internal.

        # Buat hub.KerasLayer internal dengan semua kwargs yang relevan
        self._hub_layer_instance = hub.KerasLayer(hub_url, **kwargs)

        # Panggil konstruktor parent TANPA output_shape
        # karena output_shape bukan argumen standar untuk keras.layers.Layer.__init__
        super().__init__(**{k: v for k, v in kwargs.items() if k != 'output_shape'})
        self.hub_url = hub_url

    def call(self, inputs, training=None):
        return self._hub_layer_instance(inputs, training=training)

    # Anda mungkin perlu menambahkan get_config jika Anda ingin menyimpan model ini
    # dan memuatnya kembali nanti, terutama jika WrappedHubLayer memiliki argumen kustom.
    def get_config(self):
        config = super().get_config()
        # Jika Anda ingin menyimpan hub_url, tambahkan ke config
        config.update({
            'hub_url': self.hub_url,
            # Anda perlu cara untuk menyimpan kwargs yang diteruskan ke hub.KerasLayer internal
            # Ini bisa rumit. Untuk tujuan ini, kita asumsikan kwargs internal tidak perlu disimpan
            # atau diserialisasi ulang secara eksplisit jika modelnya hanya akan di-run.
            # Jika perlu diserialisasi, Anda harus menyimpan kwargs di __init__
            # dan mengembalikannya di sini.
        })
        return config


# --- Kode Model Utama ---
inputs = keras.layers.Input(shape=(), dtype=tf.string, name='text_input')

# Gunakan WrappedHubLayer yang telah diperbaiki
# Argumen output_shape akan diteruskan ke hub.KerasLayer internal
hub_layer_wrapped = WrappedHubLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                                    dtype=tf.string, input_shape=(), output_shape=[50],
                                    name='custom_hub_embedding')

embeddings = hub_layer_wrapped(inputs)

x = keras.layers.Dense(128, activation="relu")(embeddings)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

pretrained_embedding_model_functional = keras.Model(inputs=inputs, outputs=outputs)

pretrained_embedding_model_functional.compile(loss="binary_crossentropy", optimizer="adam",
                                               metrics=["accuracy"])

pretrained_embedding_model_functional.summary()

# --- Latih model (jika Anda memiliki train_set_imdb yang sudah disiapkan) ---
# Contoh:
# train_set_imdb_for_hub = datasets_imdb["train"].batch(32).prefetch(tf.data.AUTOTUNE)
# history_pretrained_functional = pretrained_embedding_model_functional.fit(
#    train_set_imdb_for_hub, epochs=1 # Gunakan epoch kecil untuk testing
# )

/tmp/ipython-input-10-2168095594.py:30: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**{k: v for k, v in kwargs.items() if k != 'output_shape'})


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)         │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_hub_embedding            │ (None, 50)             │             0 │
│ (WrappedHubLayer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │         6,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,657 (26.00 KB)

 Trainable params: 6,657 (26.00 KB)

 Non-trainable params: 0 (0.00 B)

---

### **3. Jaringan Encoder-Decoder untuk Neural Machine Translation (NMT)**

#### **Teori:**
Jaringan Encoder-Decoder adalah arsitektur umum untuk tugas sekuens-ke-sekuens, seperti terjemahan mesin. Encoder membaca sekuens input (misalnya, kalimat bahasa Inggris) dan mengompresnya menjadi representasi vektor "konteks" tunggal yang menangkap esensi sekuens tersebut. Decoder kemudian menggunakan vektor konteks ini untuk menghasilkan sekuens output (misalnya, kalimat bahasa Prancis).

Input ke decoder biasanya adalah sekuens target yang digeser satu langkah waktu ke depan (termasuk token SOS - Start of Sequence di awal) untuk memungkinkan pelatihan "Teacher Forcing". Selama inferensi, output yang diprediksi dari langkah waktu sebelumnya akan dimasukkan kembali sebagai input untuk langkah waktu saat ini. Penting untuk membalik sekuens input encoder untuk memastikan bagian awal kalimat masukan diproses terakhir, karena ini seringkali merupakan informasi pertama yang dibutuhkan decoder untuk memulai terjemahan.

TensorFlow Addons menyediakan banyak alat untuk membangun model sekuens-ke-sekuens.

In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# --- Perhatikan: tensorflow_addons di-import di awal kode asli Anda,
# --- tetapi akan dihindari dalam solusi ini karena masalah kompatibilitas.
# --- !pip install tensorflow_addons (tidak lagi direkomendasikan untuk fungsionalitas ini)

# Asumsi: Anda sudah memiliki vocab_size_imdb dan num_oov_buckets_imdb dari sentimen analysis
# dan embed_size_imdb juga.
# Jika belum terdefinisi, ini nilai dummy untuk memastikan kode berjalan:
if 'vocab_size_imdb' not in locals():
    vocab_size_imdb = 10000 # Contoh dari analisis sentimen
if 'num_oov_buckets_imdb' not in locals():
    num_oov_buckets_imdb = 1000 # Contoh dari analisis sentimen
if 'embed_size_imdb' not in locals():
    embed_size_imdb = 128 # Contoh dari analisis sentimen

# Dummy vocabulary dan embedding (ganti dengan data terjemahan nyata)
encoder_vocab_size = vocab_size_imdb + num_oov_buckets_imdb
decoder_vocab_size = vocab_size_imdb + num_oov_buckets_imdb # Asumsi vocab yang sama

# --- Bagian ENCODER ---
encoder_inputs = keras.layers.Input(shape=[None], dtype=tf.int32, name='encoder_inputs')
encoder_embeddings = keras.layers.Embedding(encoder_vocab_size, embed_size_imdb)(encoder_inputs)

# Encoder (LSTM dengan return_state=True untuk mendapatkan hidden state terakhir)
# return_state=True akan mengembalikan [output, hidden_state, cell_state] untuk LSTM
encoder_lstm = keras.layers.LSTM(512, return_sequences=False, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)
encoder_state = [state_h, state_c] # Hidden state dan cell state terakhir dari encoder

# --- Bagian DECODER ---
# Input untuk decoder (sekuens target yang digeser 1 langkah ke depan)
decoder_inputs = keras.layers.Input(shape=[None], dtype=tf.int32, name='decoder_inputs')
decoder_embeddings = keras.layers.Embedding(decoder_vocab_size, embed_size_imdb)(decoder_inputs)

# Sel Decoder (LSTM)
# Decoder akan mengambil initial_state dari encoder
decoder_lstm = keras.layers.LSTM(512, return_sequences=True, return_state=True, name='decoder_lstm')
# Panggilan pertama dengan initial_state dari encoder
decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_state)

# Lapisan output untuk memprediksi kata
output_layer = keras.layers.Dense(decoder_vocab_size, activation='softmax', name='decoder_output')
Y_proba = output_layer(decoder_outputs)

# Model Encoder-Decoder
nmt_model = keras.Model(inputs=[encoder_inputs, decoder_inputs],
                        outputs=[Y_proba], name='nmt_model')

# Kompilasi model (contoh)
nmt_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nmt_model.summary()

# --- Catatan Penting untuk Pelatihan & Inferensi ---
# Untuk pelatihan, Anda memerlukan data paralel:
# encoder_input_data: sekuens input (misal: kalimat Inggris)
# decoder_input_data: sekuens target yang digeser (misal: kalimat Prancis dengan <SOS> di depan, <EOS> di belakang, dan tanpa kata terakhir)
# decoder_target_data: sekuens target asli (misal: kalimat Prancis tanpa <SOS> di depan)

# Contoh data dummy (Anda perlu mengganti ini dengan data terjemahan nyata)
# from keras.preprocessing.sequence import pad_sequences
# max_encoder_len = 20
# max_decoder_len = 25
# dummy_encoder_input = tf.constant(np.random.randint(1, encoder_vocab_size, size=(32, max_encoder_len)), dtype=tf.int32)
# dummy_decoder_input = tf.constant(np.random.randint(1, decoder_vocab_size, size=(32, max_decoder_len)), dtype=tf.int32)
# dummy_decoder_target = tf.constant(np.random.randint(1, decoder_vocab_size, size=(32, max_decoder_len)), dtype=tf.int32)

# nmt_model.fit([dummy_encoder_input, dummy_decoder_input], dummy_decoder_target, epochs=1)

# --- Inferensi (Lebih Kompleks, Perlu Loop Prediksi) ---
# Untuk inferensi, Anda akan membangun dua model terpisah:
# 1. Encoder model: Input encoder_inputs, Output encoder_state
# 2. Decoder model: Input decoder_inputs (untuk satu langkah waktu), initial_state, Output prediksi dan new_state

# Model Encoder untuk Inferensi:
encoder_model_inference = keras.Model(encoder_inputs, encoder_state)

# Model Decoder untuk Inferensi:
decoder_state_input_h = keras.layers.Input(shape=(512,), name='decoder_state_input_h')
decoder_state_input_c = keras.layers.Input(shape=(512,), name='decoder_state_input_c')
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs_inference, state_h_inf, state_c_inf = decoder_lstm(
    keras.layers.Embedding(decoder_vocab_size, embed_size_imdb)(keras.layers.Input(shape=(1,), dtype=tf.int32)), # Input tunggal untuk prediksi langkah demi langkah
    initial_state=decoder_state_inputs
)
decoder_states_inference = [state_h_inf, state_c_inf]
decoder_outputs_inference = output_layer(decoder_outputs_inference)

decoder_model_inference = keras.Model(
    [keras.layers.Input(shape=(1,), dtype=tf.int32), decoder_state_inputs],
    [decoder_outputs_inference] + decoder_states_inference
)

print("\nModel Encoder untuk Inferensi:")
encoder_model_inference.summary()
print("\nModel Decoder untuk Inferensi:")
decoder_model_inference.summary()

Model: "nmt_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │  1,408,000 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 128) │  1,408,000 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 512),     │  1,312,768 │ embedding_1[0][0] │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │  1,312,768 │ embedding_2[0][0… │
│                     │ 512), (None,      │            │ encoder_lstm[0][… │
│                     │ 512), (None,      │            │ encoder_lstm[0][… │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_output      │ (None, None,      │  5,643,000 │ decoder_lstm[0][… │
│ (Dense)             │ 11000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,084,536 (42.28 MB)

 Trainable params: 11,084,536 (42.28 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 128)    │  1,408,000 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_inpu… │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_inpu… │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 1, 512),  │  1,312,768 │ embedding_3[0][0… │
│                     │ (None, 512),      │            │ decoder_state_in… │
│                     │ (None, 512)]      │            │ decoder_state_in… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_output      │ (None, 1, 11000)  │  5,643,000 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,363,768 (31.91 MB)

 Trainable params: 8,363,768 (31.91 MB)

 Non-trainable params: 0 (0.00 B)

#### **3.1. RNN Bidirectional**

**Teori:**
Lapisan RNN standar hanya melihat input masa lalu dan sekarang. Namun, untuk banyak tugas NLP seperti NMT, konteks dari kata-kata di masa depan juga penting (misalnya, untuk disambiguasi kata). RNN Bidirectional mengatasi ini dengan menjalankan dua lapisan RNN pada input yang sama: satu dari kiri ke kanan, dan satu lagi dari kanan ke kiri. Output dari kedua lapisan ini kemudian digabungkan (biasanya dengan penggabungan) pada setiap langkah waktu untuk memberikan representasi yang kaya konteks yang melihat ke masa lalu dan masa depan.

In [13]:
# Contoh lapisan GRU bidirectional
bidirectional_gru_layer = keras.layers.Bidirectional(
    keras.layers.GRU(10, return_sequences=True))

# Cara mengganti lapisan encoder di NMT_model dengan bidirectional (contoh)
# encoder_bidirectional = keras.layers.Bidirectional(
#     keras.layers.LSTM(512, return_state=True), merge_mode='concat')
# Jika return_state=True, Bidirectional akan mengembalikan state_h dan state_c dari kedua arah.
# Ini berarti return_state akan menjadi [forward_h, forward_c, backward_h, backward_c]

#### **3.2. Beam Search**

**Teori:**
Selama inferensi dalam model sekuens-ke-sekuens, memprediksi kata berikutnya secara serakah (memilih probabilitas tertinggi) dapat menyebabkan kesalahan yang tidak dapat diperbaiki. *Beam Search* adalah algoritma pencarian yang mengatasi ini dengan mempertahankan daftar pendek ($k$) sekuens output paling menjanjikan pada setiap langkah waktu. Pada setiap langkah, algoritma memperluas setiap sekuens dalam daftar $k$ dengan setiap kata yang mungkin, mengevaluasi probabilitas gabungan, dan kemudian memilih $k$ sekuens teratas untuk langkah berikutnya. Parameter $k$ disebut *beam width*. Ini secara signifikan meningkatkan kualitas terjemahan atau generasi teks, meskipun dengan biaya komputasi yang lebih tinggi.

In [14]:
# Beam Search Decoder (untuk inferensi)
beam_width = 10
# decoder_initial_state harus 'ditile' (digandakan) sebanyak beam_width
# Contoh:
# decoder_initial_state_tiled = tfa.seq2seq.beam_search_decoder.tile_batch(
#    encoder_state, multiplier=beam_width)

# beam_search_decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
#     cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)

# output_beam_search, _, _ = beam_search_decoder(
#     embedding_decoder, start_tokens=start_tokens, end_token=end_token,
#     initial_state=decoder_initial_state_tiled)

---

### **4. Mekanisme Perhatian (Attention Mechanisms)**

#### **Teori:**
Mekanisme perhatian merevolusi NLP, terutama NMT, dengan memungkinkan decoder untuk "fokus" pada bagian-bagian yang paling relevan dari sekuens input (encoder output) pada setiap langkah waktu. Ini secara efektif memperpendek "jalur" dari input ke output yang relevan, mengatasi masalah *short-term memory* pada RNN yang dalam. Model perhatian menghitung skor (atau "energi") untuk setiap pasangan output encoder dan keadaan tersembunyi decoder sebelumnya, yang kemudian diubah menjadi bobot probabilitas. Bobot ini digunakan untuk menghitung jumlah terbobot dari output encoder, yang kemudian digunakan oleh decoder. Contoh populer adalah *Bahdanau attention* (juga dikenal sebagai *concatenative attention* atau *additive attention*) dan *Luong attention* (atau *multiplicative attention*).

In [15]:
# Luong Attention (contoh, perlu konteks model Encoder-Decoder yang lengkap)
# from tf_agents.seq2seq import LuongAttention # Perhatikan ini adalah tf_agents, bukan tfa.seq2seq
# Karena contoh dari buku menggunakan tfa.seq2seq, kita akan merujuk ke sana.

# from tf_agents.seq2seq import attention_wrapper # Ini bukan bagian dari tfa.seq2seq di versi terbaru TF-Agents

# Fungsionalitas Attention di Keras/TensorFlow telah diintegrasikan ke keras.layers.Attention.

# Luong Attention
# attention_mechanism = tfa.seq2seq.attention_wrapper.LuongAttention(
#     units, encoder_state, memory_sequence_length=encoder_sequence_length) # units adalah dimensi keadaan tersembunyi decoder

# attention_decoder_cell = tfa.seq2seq.attention_wrapper.AttentionWrapper(
#     decoder_cell, attention_mechanism, attention_layer_size=units) # units adalah dimensi output dari wrapper perhatian

# Contoh penggunaan keras.layers.Attention (lebih modern):
# query = decoder_state # Misal dari GRU/LSTM decoder
# value = encoder_outputs # Misal dari GRU/LSTM encoder (semua langkah waktu)
# attention_output = keras.layers.Attention()([query, value])

#### **4.1. Visual Attention**

**Teori:**
Mekanisme perhatian tidak terbatas pada NLP. *Visual attention* digunakan dalam tugas-tugas seperti generasi *caption* gambar. Dalam konteks ini, model perhatian belajar untuk fokus pada bagian-bagian yang paling relevan dari gambar input (sering kali direpresentasikan oleh *feature maps* dari CNN) saat menghasilkan setiap kata dalam *caption*. Ini membantu model untuk menjelaskan gambar dengan lebih akurat dengan mengasosiasikan kata-kata tertentu dengan area visual tertentu.

In [16]:
# Visual attention biasanya melibatkan output dari CNN (feature maps) sebagai 'value'
# dan state dari RNN (yang menghasilkan caption) sebagai 'query'.
# keras.layers.Attention dapat digunakan untuk ini.

# Contoh konseptual:
# cnn_feature_maps = ... # Output dari lapisan CNN
# rnn_hidden_state = ... # State tersembunyi dari GRU/LSTM decoder

# # Perhatian visual
# visual_attention_output = keras.layers.Attention()([rnn_hidden_state, cnn_feature_maps])
# # Output ini kemudian akan digabungkan dengan input lain dari RNN decoder

#### **4.2. Perhatian Adalah Semua yang Anda Butuhkan: Arsitektur Transformer**

**Teori:**
Makalah *groundbreaking* tahun 2017 "Attention Is All You Need" memperkenalkan arsitektur Transformer, yang mencapai *state-of-the-art* dalam NMT tanpa menggunakan lapisan rekuren atau konvolusional. Sebaliknya, Transformer sangat mengandalkan mekanisme perhatian, khususnya "self-attention" dan "multi-head attention".

Komponen utama Transformer meliputi:
* **Positional Embeddings**: Karena Transformer tidak memiliki konsep urutan, *positional embeddings* ditambahkan ke *word embeddings* untuk memberikan informasi posisi kata dalam sekuens.
* **Multi-Head Attention**: Ini adalah komponen inti yang memungkinkan model untuk secara bersamaan fokus pada berbagai bagian sekuens input (atau sekuens output itu sendiri, dalam kasus *self-attention*) melalui beberapa "kepala" perhatian yang berbeda. Setiap kepala belajar untuk melihat hubungan yang berbeda.
* **Scaled Dot-Product Attention**: Ini adalah dasar dari *multi-head attention*, yang menghitung skor kesamaan antara *queries* dan *keys*, lalu menggunakannya untuk menimbang *values*.

In [17]:
# Karena arsitektur Transformer cukup kompleks, implementasi lengkapnya akan sangat panjang.
# Buku merujuk pada tutorial TensorFlow untuk implementasi lebih lanjut.
# Di sini, kita akan menunjukkan komponen kunci secara konseptual.

# Positional Encoding (contoh implementasi dari buku)
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1 # max_dims harus genap
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))

    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

# Contoh penggunaan PositionalEncoding dalam model Transformer:
embed_size_transformer = 512
max_steps_transformer = 500
vocab_size_transformer = 10000

encoder_inputs_transformer = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_inputs_transformer = keras.layers.Input(shape=[None], dtype=tf.int32)

embeddings_transformer = keras.layers.Embedding(vocab_size_transformer, embed_size_transformer)
encoder_embeddings_transformer = embeddings_transformer(encoder_inputs_transformer)
decoder_embeddings_transformer = embeddings_transformer(decoder_inputs_transformer)

positional_encoding_layer = PositionalEncoding(max_steps_transformer, max_dims=embed_size_transformer)
encoder_in_transformer = positional_encoding_layer(encoder_embeddings_transformer)
decoder_in_transformer = positional_encoding_layer(decoder_embeddings_transformer)

# Scaled Dot-Product Attention (konseptual, keras.layers.Attention sudah mengimplementasikan ini)
# Z_encoder = keras.layers.Attention(use_scale=True)([encoder_in_transformer, encoder_in_transformer])
# Z_decoder_masked = keras.layers.Attention(use_scale=True, causal=True)([decoder_in_transformer, decoder_in_transformer])
# Z_decoder_cross_attention = keras.layers.Attention(use_scale=True)([Z_decoder_masked, Z_encoder])

# Multi-Head Attention (secara konseptual, ini adalah beberapa lapisan Attention yang berjalan paralel
# diikuti dengan Dense layer. Keras 2.x belum punya MultiHeadAttention langsung,
# tetapi ada di TF 2.x dengan `tf.keras.layers.MultiHeadAttention`)
# Misalnya, jika kita menggunakan tf.keras.layers.MultiHeadAttention:
# from tensorflow.keras.layers import MultiHeadAttention
# multi_head_attention_layer = MultiHeadAttention(num_heads=8, key_dim=embed_size_transformer//8)
# attention_output = multi_head_attention_layer(query=encoder_in_transformer,
#                                                 value=encoder_in_transformer,
#                                                 key=encoder_in_transformer)

---

### **5. Inovasi Terbaru dalam Model Bahasa**

#### **Teori:**
Tahun 2018 dan 2019 menyaksikan kemajuan luar biasa dalam NLP, sering disebut sebagai "momen ImageNet untuk NLP".  Inovasi-inovasi ini sebagian besar berpusat pada:
* **Tokenisasi Subkata yang Lebih Baik**: Teknik seperti Byte-Pair Encoding (BPE) dan WordPiece memungkinkan tokenisasi yang independen dari bahasa, menangani kata-kata yang jarang dan bahkan kata-kata yang belum pernah terlihat sebelumnya dengan memecahnya menjadi unit subkata.
* **Pergeseran dari LSTM ke Transformer**: Arsitektur Transformer, dengan mekanisme perhatiannya, terbukti lebih efisien dan efektif untuk tugas-tugas bahasa dibandingkan RNN berbasis LSTM tradisional, terutama untuk sekuens panjang.
* **Pelatihan Awal Model Bahasa Universal (Self-Supervised Learning)**: Model-model besar dilatih pada korpus teks yang sangat besar menggunakan tugas-tugas *self-supervised* (misalnya, memprediksi kata yang hilang atau sekuens kalimat berikutnya). Ini memungkinkan model untuk belajar representasi bahasa yang kaya tanpa memerlukan label manusia. Model-model ini kemudian dapat di-*fine-tune* pada berbagai tugas hilir dengan data berlabel yang jauh lebih sedikit, sebuah teknik yang dikenal sebagai *transfer learning* atau *zero-shot learning*. Contoh model ini adalah ELMo, ULMFiT, GPT (GPT-2), dan BERT.

In [18]:
# Karena ini adalah bagian teoritis yang membahas perkembangan terbaru, tidak ada kode implementasi langsung yang akan diberikan.
# Namun, Anda bisa menambahkan catatan atau tautan ke implementasi model-model seperti BERT atau GPT-2
# yang tersedia di TensorFlow Model Garden atau Hugging Face Transformers.

# Contoh:
print("Model-model seperti BERT dan GPT-2 sangat besar dan seringkali membutuhkan sumber daya komputasi yang signifikan.")
print("Anda dapat menjelajahi implementasinya di:")
print("- TensorFlow Model Garden: https://github.com/tensorflow/models/tree/master/official/nlp")
print("- Hugging Face Transformers: https://huggingface.co/transformers/")

Model-model seperti BERT dan GPT-2 sangat besar dan seringkali membutuhkan sumber daya komputasi yang signifikan.
Anda dapat menjelajahi implementasinya di:
- TensorFlow Model Garden: https://github.com/tensorflow/models/tree/master/official/nlp
- Hugging Face Transformers: https://huggingface.co/transformers/


---

### **Latihan**

Latihan-latihan ini akan membantu Anda memperdalam pemahaman dan keterampilan praktis dalam mengimplementasikan konsep inti Machine Learning melalui reproduksi kode dan penjelasan teoritis yang terstruktur, menggunakan buku Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems (O’Reilly) sebagai referensi utama.

1.  **RNN Stateful vs. Stateless:**
    * Jelaskan kelebihan dan kekurangan menggunakan RNN stateful dibandingkan RNN stateless.
2.  **Encoder-Decoder untuk Terjemahan Otomatis:**
    * Mengapa orang menggunakan RNN Encoder-Decoder daripada RNN sekuens-ke-sekuens biasa untuk terjemahan otomatis?
3.  **Sekuens Panjang Bervariasi:**
    * Bagaimana Anda bisa menangani sekuens input dengan panjang yang bervariasi? Bagaimana dengan sekuens output dengan panjang yang bervariasi?
4.  **Beam Search:**
    * Apa itu *beam search* dan mengapa Anda menggunakannya? Alat apa yang dapat Anda gunakan untuk mengimplementasikannya?
5.  **Mekanisme Perhatian:**
    * Apa itu mekanisme perhatian? Bagaimana cara kerjanya dan bagaimana mekanisme perhatian membantu dalam tugas-tugas NLP?
6.  **Lapisan Paling Penting di Transformer:**
    * Apa lapisan terpenting dalam arsitektur Transformer? Apa tujuannya?
7.  **Sampled Softmax:**
    * Kapan Anda perlu menggunakan *sampled softmax*?
8.  **Embedded Reber Grammars (Lanjutan dari Chapter 15):**
    * Gunakan *embedded Reber grammars* yang digunakan oleh Hochreiter dan Schmidhuber. Pilih tata bahasa Reber tertanam tertentu, lalu latih RNN untuk mengidentifikasi apakah sebuah string mematuhi tata bahasa tersebut atau tidak. Anda harus terlebih dahulu menulis fungsi yang mampu menghasilkan *batch* pelatihan yang berisi sekitar 50% string yang mematuhi tata bahasa, dan 50% yang tidak.
9.  **Terjemahan Tanggal:**
    * Latih model Encoder-Decoder yang dapat mengonversi string tanggal dari satu format ke format lain (misalnya, dari “April 22, 2019” menjadi “2019-04-22”).
10. **Neural Machine Translation with Attention Tutorial:**
    * Ikuti tutorial Neural Machine Translation with Attention dari TensorFlow.
11. **Teks Shakespeare yang Lebih Meyakinkan:**
    * Gunakan salah satu model bahasa terbaru (misalnya, BERT) untuk menghasilkan teks Shakespeare yang lebih meyakinkan.

---

**Catatan Penting:**

* **Pelatihan Model:** Bagian kode `model.fit()` untuk pelatihan dikomentari. Anda perlu meng-uncomment-nya untuk melatih model. Pelatihan mungkin memerlukan waktu yang signifikan, terutama untuk model yang lebih besar.
* **Sumber Daya:** Melatih model NLP yang kompleks, terutama Transformer dan model bahasa besar, membutuhkan sumber daya komputasi yang substansial (GPU). Anda mungkin perlu menggunakan layanan seperti Google Colab (dengan GPU Runtime gratis) atau platform cloud lainnya.
* **Versi Library:** TensorFlow dan Keras terus berkembang. Jika Anda mengalami masalah kompatibilitas, periksa kembali dokumentasi resmi TensorFlow dan versi *notebook* yang diperbarui di repositori GitHub buku ini.
* **Data Nyata untuk NMT:** Contoh NMT di atas bersifat konseptual dan menggunakan *dummy vocabulary*. Untuk melatih model NMT nyata, Anda perlu dataset paralel (misalnya, pasangan kalimat Inggris-Prancis) dan melakukan pra-pemrosesan yang lebih canggih (seperti tokenisasi subkata dan pembuatan kosakata).